# Build a language model based on wikipedia in you language 

The notebooke includes the whole process but you will need to help the process if the following problem happens:  
-you internet connection is interrupted (stage 1)

-you run out of diskspace

-because of the huge memory consumption. creatig the databunch for the training requires lots of memory


In order ot handle these problemn the notebook is divided into stages. If the process is failes in a stage then you can resume the processing from the beginning of that stage by:
-restarting the kernel "Kernel"/Restart & Clear Output

-running the cells in stage 0: initialization

-running the cells from the start of the stage that failed 

# Stage 0: Initialisation
lang: You must set the parameter "lang" to the language you want to build a model for. Fx:

fr: for french

en: for english

de:for german

da: for danish 

etc.

pathData: You must se the location where you want your data stored using the parameter pathData. Consider using a ssd-rive

In [1]:
lang="en"

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.text import * 

from IPython.display import display, HTML
from pathlib import *
import re
import shutil

In [4]:
pathData       = Path("../nlp-data")
path           = pathData / lang
pathDump       = path/"wiki-dump"
pathJson       = path/"wiki-json"
pathTrainValid = path/"wiki-train_valid"
spCache        = "sp-model"
pathSPVocab    = pathTrainValid / spCache

# Stage 1: download the selected language from wikipedia and convert the articles to json

In [ ]:
fn = f"{lang}wiki-latest-pages-articles.xml.bz2"
if len(list(pathDump.glob("*.bz2")))==0:
    pathDump.mkdir(parents=True,exist_ok=True)
    url  = f"https://dumps.wikimedia.org/{lang}wiki/latest/{fn}"
    pDist = str(pathData/lang)
    !wget -c $url -P $pDist



# Stage 2: Convert wikipedia dump to articles in json 
minWords: wikipedia sections with fewer than minWords are not converte. If you language does not use space " " to "separate" words then set this prameter to 1

In [ ]:
#import json as json
pathWikiExtractor = Path("../wikiextractor")

In [ ]:
!git clone https://github.com/attardi/wikiextractor.git $pathWikiExtractor

In [ ]:
cmd = f"cd {str(pathWikiExtractor)} && python setup.py install"
! $cmd

In [ ]:
%%time
#extracting the json-files. This takes about 1 hour for french with files read from and saved to a ssd hardrive 
fn_wikidump = list(pathDump.iterdir())[0]
strcmd = f"cd {str(pathWikiExtractor)} && WikiExtractor.py -o {str(pathJson)} --json -q  {str(fn_wikidump)}"
print(strcmd)
!$strcmd

# Stage 3: building a vocabulary using sentencepiece

Now we separate the title of the wikipedia section from text section that we keep. 

In order to makes a first reduction on the number of section we clean the text with the preprocessing rules from fastai and ignore text with less than "minWords"

You must set the lenght of the shortes sections you want to keep using the parameter "minWords"

In [8]:
minWords  = 25
chunksize = 25000

In [9]:
!pip install sentencepiece

In [10]:
path       = Path("../../data/nlp-data/fr/")
cache_name = "sp-model"

In [11]:
class SentencepieceWikiModel:
    def __init__(self, lang:str, pathJson:Path, pathTrainValid:Path, cache_name:str='sp-model', 
                 vocab_size:int=32000, model_type:str='unigram', 
                 rules=text.transform.default_pre_rules ):  #should include removal of repetitions
        self.lang           = lang
        self.pathJson       = pathJson
        self.pathTrainValid = pathTrainValid
        self.pathVocab      = self.pathTrainValid / cache_name
        self.vocab_size     = vocab_size
        self.model_type     = model_type
        self.rules          = rules
        
        self.pathVocab.mkdir(parents=True, exist_ok=True)

    def wikijson2TrainingData(self):
        
        def save_sections( f_out, data, fileCount ):
            p = pathTxt / f"{fileCount}.txt"
            with p.open("w+") as fw:
                fw.write( "\n".join(data["text"]) )
            data.to_csv(f_out, index=False, header=False, mode='a')
        
        "generate text files for training af sentencepiece vocabulary " \
        "and a csv-file for training a languagemodel with the vocabulary and the wiki-text in the csv-file"
        pathTxt  = self.pathTrainValid/"txt"
        print(pathTxt)
        pathTxt.mkdir(parents=True, exist_ok=True)
        
        pathParts = len(pathJson.parts)        
        pathcsv   = self.pathTrainValid/'wiki.csv'
        with pathcsv.open("w") as f_out:
            fileCount = 0
            fields    = ['text', 'wordcount']
            data      = pd.DataFrame(columns=fields)
            data.to_csv(f_out, index=False, mode='a')
            
            sections = []
            for fn in self.pathJson.glob("**/wiki*"):
                with open(fn, encoding='utf-8') as f:
                    for line in f:
                        section = json.loads(line)
                        
                        if section['text'].find(section['title']) >=0 :
                            section['text'] = section['text'][len(section['title'])+2:]
                
                        section['text']      = section['text'].replace('\n\n', "\n")
                        section['text']      = reduce(lambda t, rule: rule(t), self.rules, section['text'])
                        section['wordcount'] = len(re.findall(r'\w+',section['text']))
            
                        if section["wordcount"] > minWords:
                            sections.append({k:section[k] for k in fields})
                
                if len(sections) > chunksize: 
                    data = pd.DataFrame(sections)
                    sections = []
                    
                    save_sections(f_out, data, fileCount)
                    fileCount += 1
                    data = None
                    
            if len(sections) > 0: 
                data = pd.DataFrame(sections)
                sections = []
                    
                save_sections(f_out, data, fileCount)
                fileCount += 1
                data = None
                
            
    def getUserdefinedSymbols(self): 
        return [text.transform.BOS,
                text.transform.PAD,
                text.transform.TK_MAJ,
                text.transform.TK_UP,
                text.transform.TK_REP,
                text.transform.TK_WREP,
                text.transform.FLD ] 

    def trainVocabulary(self): 
        pathSrcTxt   = self.pathTrainValid / "txt"
        model_prefix = self.pathVocab / "m"
    
        #Set the following controls to sentencepiece values until there is a release where we can set the token value
        #Note taku910 has already made the change but the pip of sentencepiewce version has not been updated 
        text.transform.UNK = "<unk>"
        #text.transform.BOS = "<s>"
        #text.transform.PAD = "<pad>"
    
        #create control ids for the rest of the fastai control tokens in case the user needs them
        #it is the responsibility of fastai to generate and use the control tokens them and apply them before decoding
        #Fx applying TK_MAJ after tokenization would change She to two token TK_MAJ+she.
        #Problem! Sentencepiece would tokenize "Elle" as _Elle so our deal_caps would not catch it
        str_specialcases = ",".join(getUserdefinedSymbols()) 
    
        pathSrc_list = [str(s) for s in pathSrcTxt.glob("**/*.txt")]
        pathSrc_list= ",".join(getUserdefinedSymbols)
    
        sp_params = f"--input={pathSrc_list} "  \
                    f"--bos_id=-1 " \
                    f"--eos_id=-1 " \
                    f"--pad_id=-1 " \
                    f"--user_defined_symbols={str_specialcases} " \
                    f"--character_coverage=1.0 " \
                    f"--model_prefix={model_prefix} " \
                    f"--vocab_size={self.vocab_size} " \
                    f"--model_type={self.model_type} " 
    
        #f"--split_by_number=1 " \
        #hard_vocab_limit=False
        #use_all_vocab
        #print(sp_params)
        import sentencepiece as spm
        spm.SentencePieceTrainer.Train(sp_params)
        
        #convert sentencepieces vocabulary to a format fastai can read
        with open( self.pathVocab/"m.vocab", 'r') as f:
            vocab = [line.split('\t')[0] for line in f.readlines()]
        pickle.dump(vocab, open( self.pathVocab / "itos.pkl", "wb"))



In [12]:
class SentencepieceTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        path,cache_name = pathTrainValid, "sp-model"
    #def __init__(self, path:PathOrStr, cache_name:str='sp-model'):
        self.pathVocab = path / cache_name
        self.vocab_    = Vocab(pickle.load(open(self.pathVocab/'itos.pkl', 'rb')))
        self.tok       = spm.SentencePieceProcessor()
        
        self.tok.Load(str(Path(path) / cache_name / 'm.model'))
        text.transform.UNK = "<unk>"

    def tokenizer(self, t:str) -> List[str]:
        return self.tok.EncodeAsPieces(t)
    
    def add_special_cases(self, toks:Collection[str]):
        #this should have been done when training sentencepiece
        pass
    
    def vocab(self): return self.vocab_

In [13]:
swm = SentencepieceWikiModel(lang=lang, pathJson=pathJson, pathTrainValid=pathTrainValid)

%time swm.wikijson2TrainingData()   

../nlp-data/en/wiki-train_valid/txt
CPU times: user 46min 17s, sys: 48.7 s, total: 47min 6s
Wall time: 48min 9s


In [ ]:
swm = SentencepieceWikiModel(lang=lang, pathJson=pathJson, pathTrainValid=pathTrainValid)
%time swm.trainVocabulary()

In [ ]:
swm = SentencepieceWikiModel(lang=lang, pathJson=pathJson, pathTrainValid=pathTrainValid)
sp = spm.SentencePieceProcessor()
sp.Load(str(pathTrainValid / cache_name / "m.model"))
print("1: Size of vocabulary:",sp.GetPieceSize())
sentence = "She is tall. He is small"
print("2:", sp.EncodeAsPieces(sentence))
print("3:", sp.EncodeAsIds(sentence))


In [ ]:
print(f"Control symbol")
for s in ["<unk>"]: print(f"{s}({sp.PieceToId(s)})")

print(f"\nuser_defined_symbols")
for s in swm.getUserdefinedSymbols():print(f"{s}({sp.PieceToId(s)})")
